<a href="https://colab.research.google.com/github/Vahid-Kh/NLP/blob/main/gpt_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Building a GPT

Companion notebook to the [Zero To Hero](https://karpathy.ai/zero-to-hero.html) video on GPT.

In [135]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://github.com/Vahid-Kh/NLP/blob/da12adf98d303843e554adc71a4bd9ec1814bd93/test.txt

--2023-02-22 11:51:20--  https://github.com/Vahid-Kh/NLP/blob/da12adf98d303843e554adc71a4bd9ec1814bd93/test.txt
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘test.txt.1’

test.txt.1              [  <=>               ] 424.85K  1.73MB/s    in 0.2s    

2023-02-22 11:51:20 (1.73 MB/s) - ‘test.txt.1’ saved [435045]



In [149]:
# read it in to inspect it
with open('/content/input.txt.1', 'r', encoding='utf-8') as f:
    text = f.read()

In [150]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  4042155


In [173]:
# let's look at the first 1000 characters
print(text[:1000])
import re
text = re.sub(r'\W+', ' ', text)
chars_to_drop = "ª¼ÂÃÎâ"
pattern = "[" + re.escape(chars_to_drop) + "]"
text = re.sub(pattern, "", text)


description HUMAN NECESSITIES AGRICULTURE FORESTRY ANIMAL HUSBANDRY HUNTING TRAPPING FISHING SOIL WORKING IN AGRICULTURE OR FORESTRY PARTS DETAILS OR ACCESSORIES OF AGRICULTURAL MACHINES OR IMPLEMENTS IN GENERAL Hand tools Spades Shovels with teeth Hoes Hand cultivators with a single blade with two or more blades with blades provided with teeth with teeth only Tools for uprooting weeds Tong like tools Combinations of different kinds of hand tools Attaching the blades or the like to handles Interchangeable or adjustable blades for treating meadows or lawns Ploughs Ploughs with fixed plough shares Man driven ploughs Animal drawn ploughs without alternating possibility i e incapable of making an adjacent furrow on return journey Swing ploughs Trussed beam ploughs Single wheel ploughs Two wheel beam ploughs Frame ploughs Alternating ploughs i e capable of making an adjacent furrow on return journey Turn wrest ploughs Balance ploughs with parallel plough units used alternately Tractor drawn

In [174]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

 0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
63


In [175]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[44, 45, 45, 0, 56, 44, 41, 54, 41]
hii there


In [176]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([3973418]) torch.int64
tensor([40, 41, 55, 39, 54, 45, 52, 56, 45, 51, 50,  0, 18, 31, 23, 11, 24,  0,
        24, 15, 13, 15, 29, 29, 19, 30, 19, 15, 29,  0, 11, 17, 28, 19, 13, 31,
        22, 30, 31, 28, 15,  0, 16, 25, 28, 15, 29, 30, 28, 35,  0, 11, 24, 19,
        23, 11, 22,  0, 18, 31, 29, 12, 11, 24, 14, 28, 35,  0, 18, 31, 24, 30,
        19, 24, 17,  0, 30, 28, 11, 26, 26, 19, 24, 17,  0, 16, 19, 29, 18, 19,
        24, 17,  0, 29, 25, 19, 22,  0, 33, 25, 28, 21, 19, 24, 17,  0, 19, 24,
         0, 11, 17, 28, 19, 13, 31, 22, 30, 31, 28, 15,  0, 25, 28,  0, 16, 25,
        28, 15, 29, 30, 28, 35,  0, 26, 11, 28, 30, 29,  0, 14, 15, 30, 11, 19,
        22, 29,  0, 25, 28,  0, 11, 13, 13, 15, 29, 29, 25, 28, 19, 15, 29,  0,
        25, 16,  0, 11, 17, 28, 19, 13, 31, 22, 30, 31, 28, 11, 22,  0, 23, 11,
        13, 18, 19, 24, 15, 29,  0, 25, 28,  0, 19, 23, 26, 22, 15, 23, 15, 24,
        30, 29,  0, 19, 24,  0, 17, 15, 24, 15, 28, 11, 22,  0, 18, 37, 50, 40,
      

In [177]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [178]:
block_size = 8
train_data[:block_size+1]

tensor([40, 41, 55, 39, 54, 45, 52, 56, 45])

In [179]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([40]) the target: 41
when input is tensor([40, 41]) the target: 55
when input is tensor([40, 41, 55]) the target: 39
when input is tensor([40, 41, 55, 39]) the target: 54
when input is tensor([40, 41, 55, 39, 54]) the target: 45
when input is tensor([40, 41, 55, 39, 54, 45]) the target: 52
when input is tensor([40, 41, 55, 39, 54, 45, 52]) the target: 56
when input is tensor([40, 41, 55, 39, 54, 45, 52, 56]) the target: 45


In [180]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[13, 25, 23, 12, 31, 29, 30, 19],
        [55,  0, 23, 51, 57, 48, 40, 55],
        [51, 54,  0, 52, 51, 48, 61, 52],
        [55,  0, 41,  0, 43,  0, 38, 61]])
targets:
torch.Size([4, 8])
tensor([[25, 23, 12, 31, 29, 30, 19, 12],
        [ 0, 23, 51, 57, 48, 40, 55,  0],
        [54,  0, 52, 51, 48, 61, 52, 41],
        [ 0, 41,  0, 43,  0, 38, 61,  0]])
----
when input is [13] the target: 25
when input is [13, 25] the target: 23
when input is [13, 25, 23] the target: 12
when input is [13, 25, 23, 12] the target: 31
when input is [13, 25, 23, 12, 31] the target: 29
when input is [13, 25, 23, 12, 31, 29] the target: 30
when input is [13, 25, 23, 12, 31, 29, 30] the target: 19
when input is [13, 25, 23, 12, 31, 29, 30, 19] the target: 12
when input is [55] the target: 0
when input is [55, 0] the target: 23
when input is [55, 0, 23] the target: 51
when input is [55, 0, 23, 51] the target: 57
when input is [55, 0, 23, 51, 57] the target: 48
when input is

In [181]:
print(xb) # our input to the transformer

tensor([[13, 25, 23, 12, 31, 29, 30, 19],
        [55,  0, 23, 51, 57, 48, 40, 55],
        [51, 54,  0, 52, 51, 48, 61, 52],
        [55,  0, 41,  0, 43,  0, 38, 61]])


In [182]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 63])
tensor(4.8644, grad_fn=<NllLossBackward0>)
 Ue8m7TYh LwajmRFdGSdcKjNW1aFGV124htCLTYFEPQ1xuWeyslIELK2PjN2 mt2ZLldYmhy8ebIqeh 05I5gN1BCFpRwj8mUigw


In [183]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [184]:
batch_size = 32
for steps in range(100): # increase number of steps for good results... 
    
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


4.4559431076049805


In [185]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))

 rdGLCzk0Dg8m UReOzpWeGZ5x1vSCw2C UuReD9BAQAdMwi9eaPQoTGufFZ3W11HvZGf9jcEPD36Ri1 MSxY7BIlIxjfkMmUR396fgWPe l 59Y2LeaPSiaVqVoAhfjBDhcnt1Q1aPUyLJb7kxXHcqrSWacOmzXKqJ UdhAHX7HLrQJBeiPeSC15gWplECHCztbjAsFECDe8zTCWRb1srVQPpIxx9r3XLc8rsw1KwawhYsOxuh2  vigQw4J57L7tE8kI25YalMm7oZfdLsle4fhNfhc6LAh tDr29REtOFf9Y8eDfTOZfMMOydhj8 tdbkrVQ35Rm3 LDg6o3hcBBoZ65GZGiaicEDtdGqYcsB8RPVIxJqVriJMtroCzeB4NdqiJ3Omt0GLcPRrk1orEkNa62pZeR8FbjfIuuQOAH08bH0u3WKtnsejxlFgWZsbO1KjBRELbqvOxJKjA1BbCUqusyPVW YRWejQPioTShYDmMD3pZle


## The mathematical trick in self-attention

In [186]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [187]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [188]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [189]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [190]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


True

In [191]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [192]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [193]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [194]:
k.var()

tensor(1.0449)

In [195]:
q.var()

tensor(1.0700)

In [196]:
wei.var()

tensor(1.0918)

In [197]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [198]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [199]:
class LayerNorm1d: # (used to be BatchNorm1d)
  
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
  
  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out
  
  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [200]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [201]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

In [202]:
# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>



### Full finished code, for reference

You may want to refer directly to the git repo instead though.

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.215405 M parameters
step 0: train loss 4.8695, val loss 4.8696
step 100: train loss 2.7018, val loss 2.7086
step 200: train loss 2.5132, val loss 2.5318
step 300: train loss 2.4199, val loss 2.4281
step 400: train loss 2.3214, val loss 2.3675
step 500: train loss 2.2242, val loss 2.2780
step 600: train loss 2.1643, val loss 2.2057
step 700: train loss 2.1039, val loss 2.1400
step 800: train loss 2.0406, val loss 2.0871
step 900: train loss 1.9917, val loss 2.0527
step 1000: train loss 1.9467, val loss 1.9944
